In [23]:
import re

def parse_log_file(log_file_path):
    total_operations = 0
    overshoot_operations = 0
    
    with open(log_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    i = 0
    while i < len(lines):
        # 检测新实验开始
        if "==== 实验 " in lines[i] and "开始 ====" in lines[i]:
            i += 1
            continue
        
        # 寻找状态-动作-试剂对部分
        if "状态-动作-试剂对:" in lines[i]:
            i += 1
            while i < len(lines) and lines[i].strip() and not "实验结束" in lines[i]:
                # 解析每一步
                match = re.match(r"  Step \d+: State = \[([^\]]+)\], Action = ([0-9.]+), Reagent = (\w+)", lines[i])
                if match:
                    total_operations += 1
                    state_str, action, reagent = match.groups()
                    state_values = [float(x) for x in state_str.split()]
                    current_ph = state_values[0]
                    target_ph = state_values[1]
                    
                    # 获取前一状态的 pH（如果存在）
                    if i > 1:
                        prev_line = lines[i - 1]
                        prev_match = re.match(r"  Step \d+: State = \[([^\]]+)\]", prev_line)
                        if prev_match:
                            prev_state_values = [float(x) for x in prev_match.group(1).split()]
                            prev_ph = prev_state_values[0]
                            # 检查过冲：前一状态与目标的乘积与当前状态与目标的乘积符号相反，且未到达目标pH附近（正负0.1）
                            if (prev_ph - target_ph) * (current_ph - target_ph) < 0 and abs(current_ph - target_ph) > 0.1:
                                overshoot_operations += 1
                
                i += 1
            i += 1  # 跳过 "实验结束" 行
        else:
            i += 1
    
    overshoot_ratio = (overshoot_operations / total_operations * 100) if total_operations > 0 else 0
    return total_operations, overshoot_operations, overshoot_ratio

# 使用示例
log_file_path = r"C:\Users\ZSY\Desktop\code\ph\强化前神经网络同样的实验.txt"  # 替换为你的日志文件路径
total_ops, overshoot_ops, overshoot_ratio = parse_log_file(log_file_path)

print(f"总实验操作数: {total_ops}")
print(f"引起过冲的实验操作数: {overshoot_ops}")
print(f"过冲比例: {overshoot_ratio:.2f}%")

总实验操作数: 38104
引起过冲的实验操作数: 10835
过冲比例: 28.44%
